In [1]:
import requests
import pprint
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

In [2]:
from api_key import flight_api_key

In [3]:
flight_num = input('Please enter your flight number (e.g., wn658): ').lower()

Please enter your flight number (e.g., wn658):  wn658


In [4]:
#free api version doesn't allow date feature filter
date = input('Please enter date of flight in YYYY-MM-DD format:')

Please enter date of flight in YYYY-MM-DD format: 2024-07-17


In [5]:
#tail number is not assigned until pulling into destination gate: 
#https://aviation.stackexchange.com/questions/16855/how-far-ahead-does-an-airline-know-which-aircraft-you-will-be-flying-on

flight_url = 'http://api.aviationstack.com/v1/flights'

params = {
  'access_key': flight_api_key,
    'flight_iata': flight_num
}

api_result = requests.get(flight_url, params)

api_response = api_result.json()

pprint.pp(api_response)

{'pagination': {'limit': 100, 'offset': 0, 'count': 2, 'total': 2},
 'data': [{'flight_date': '2024-07-17',
           'flight_status': 'landed',
           'departure': {'airport': 'Los Angeles International',
                         'timezone': 'America/Los_Angeles',
                         'iata': 'LAX',
                         'icao': 'KLAX',
                         'terminal': '1',
                         'gate': '17B',
                         'delay': 15,
                         'scheduled': '2024-07-17T06:35:00+00:00',
                         'estimated': '2024-07-17T06:35:00+00:00',
                         'actual': '2024-07-17T06:50:00+00:00',
                         'estimated_runway': '2024-07-17T06:50:00+00:00',
                         'actual_runway': '2024-07-17T06:50:00+00:00'},
           'arrival': {'airport': 'Mc Carran International',
                       'timezone': 'America/Los_Angeles',
                       'iata': 'LAS',
                       'ica

In [6]:
api_response['data'][1]['aircraft']

{'registration': 'N8903H', 'iata': 'B38M', 'icao': 'B38M', 'icao24': 'AC481D'}

In [7]:
# retrieve aircraft tail number if available: tail number is not assigned until pulling into destination gate
for i in range(len(api_response['data'])):
    aircraft = api_response['data'][i]['aircraft']
    try:
        if aircraft == None:
            print("No aircraft assigned yet, will use previous day's aircraft for same flight as proxy")
        else:
            tail_num = aircraft['registration']
    except:
        tail_num = "No aircraft information available"
print(f"aircraft's tail number is: {tail_num}")

aircraft's tail number is: N8903H


In [29]:
#cannot search with free version of api key
# plane_url = 'http://api.aviationstack.com/v1/airplanes'

# plane_result = requests.get(plane_url, params)

# plane_response = plane_result.json()

# pprint.pp(plane_response)

{'pagination': {'offset': 0, 'limit': 100, 'count': 100, 'total': 19095},
 'data': [{'id': '496471',
           'iata_type': 'B737-300',
           'airplane_id': '1',
           'airline_iata_code': '0B',
           'iata_code_long': 'B733',
           'iata_code_short': '733',
           'airline_icao_code': None,
           'construction_number': '23653',
           'delivery_date': '1986-08-21T22:00:00.000Z',
           'engines_count': '2',
           'engines_type': 'JET',
           'first_flight_date': '1986-08-02T22:00:00.000Z',
           'icao_code_hex': '4A0823',
           'line_number': '1260',
           'model_code': 'B737-377',
           'registration_number': 'YR-BAC',
           'test_registration_number': None,
           'plane_age': '31',
           'plane_class': None,
           'model_name': '737',
           'plane_owner': 'Airwork Flight Operations Ltd',
           'plane_series': '377',
           'plane_status': 'active',
           'production_line': 'Boe

# Find plane age based on tail number

## note: install webdriver manager by running this in terminal: pip install webdriver-manager

In [8]:
#note: install webdriver manager by running this in terminal: pip install webdriver-manager

# Set up the service and browser
service = ChromeService(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)

#Visiting Website
url = 'https://registry.faa.gov/aircraftinquiry/search/nnumberinquiry'
browser.get(url)

#entering the tail number
tail_num_input = browser.find_element(By.NAME, 'NNumbertxt')
tail_num_input.send_keys(tail_num)

# Find and wait for the submit button to be clickable
submit_button = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Submit')]"))
)

# Click the submit button using JavaScript to bypass potential issues
browser.execute_script("arguments[0].click();", submit_button)


In [9]:
# Wait for the results page to load
browser.implicitly_wait(10)

# Get the page source
page_source = browser.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the manufacture year using BeautifulSoup
mfr_year_elements = soup.find_all('td', {'data-label': 'Mfr Year'})

# Extract text from each element found
mfr_years = [element.text.strip() for element in mfr_year_elements]

# Close the browser
browser.quit()

In [10]:
mfr_year = int(mfr_years[0])
mfr_year

2023

In [11]:
flight_year = datetime.strptime(date, '%Y-%m-%d').year
flight_year

2024

In [12]:
plane_age = flight_year - mfr_year
plane_age

1

# Define Function

In [134]:
def aircraft_age(date, flight_num):
    from api_key import flight_api_key
    
    #retrieve flight information
    
    flight_url = 'http://api.aviationstack.com/v1/flights'
    
    params = {
      'access_key': flight_api_key,
        'flight_iata': flight_num
    }
    
    api_result = requests.get(flight_url, params)
    
    api_response = api_result.json()

    # retrieve aircraft tail number if available: tail number is not assigned until pulling into destination gate

    for i in range(len(api_response['data'])):
        aircraft = api_response['data'][i]['aircraft']
        try:
            if aircraft == None:
                print("No aircraft assigned yet, will use previous day's aircraft for same flight as proxy")
            else:
                tail_num = aircraft['registration']
        except:
            tail_num = "No aircraft information available"

    # Set up the service and browser
    service = ChromeService(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service)
    
    #Visiting Website
    url = 'https://registry.faa.gov/aircraftinquiry/search/nnumberinquiry'
    browser.get(url)
    
    #entering the tail number
    tail_num_input = browser.find_element(By.NAME, 'NNumbertxt')
    tail_num_input.send_keys(tail_num)
    
    # Find and wait for the submit button to be clickable
    submit_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Submit')]"))
    )
    
    # Click the submit button using JavaScript to bypass potential issues
    browser.execute_script("arguments[0].click();", submit_button)

    # Wait for the results page to load
    browser.implicitly_wait(10)
    
    # Get the page source
    page_source = browser.page_source
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    
    # Find the manufacture year using BeautifulSoup
    mfr_year_elements = soup.find_all('td', {'data-label': 'Mfr Year'})
    
    # Extract text from each element found
    mfr_years = [element.text.strip() for element in mfr_year_elements]
    
    # Close the browser
    browser.quit()
    
    #access mfr year as integer
    mfr_year = int(mfr_years[0])

    #extract year from date input and convert to integer
    flight_year = datetime.strptime(date, '%Y-%m-%d').year

    #calculate plane age
    plane_age = flight_year - mfr_year

    return {
        'registration/tail number': tail_num,
        "plane's manufactured year": mfr_year,
        'flight year': flight_year,
        'plane age based on flight date': plane_age
    }